In [1]:
import os
import json
from utils import read_tables, plot_all_columns
import pandas as pd
from sdv.utils import display_tables
from sdv.evaluation import evaluate
from sdmetrics.reports.multi_table import QualityReport
from sdmetrics.reports import utils
from sdv import Metadata

In [2]:
real_data = read_tables(
    '../data/rossman_store_sales/source', split_by="_", name_index=0)
synthetic_data = {}
synthetic_data['store'] = pd.read_csv('../data/rossman_store_sales/synthetic_gretelai/store_gretel_tabular_lstm.csv')

/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(
/Users/martinjurkovic/Documents/MAG1-local/Project/Synthetic-data-generation-project/notebooks/utils.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv(


In [3]:
synthetic_data['store']

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,362,a,c,340.0,NaN,NaN,0,NaN,NaN,NaN
1,865,c,c,460.0,NaN,NaN,0,NaN,NaN,NaN
2,622,a,a,520.0,NaN,NaN,0,NaN,NaN,NaN
3,621,a,a,30.0,7.0,NaN,0,NaN,NaN,NaN
4,549,d,c,2330.0,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4995,151,d,c,4360.0,10.0,2005.0,0,NaN,NaN,NaN
4996,828,d,c,3290.0,12.0,2014.0,0,NaN,NaN,NaN
4997,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN
4998,883,a,a,3200.0,6.0,2005.0,0,NaN,NaN,NaN


In [6]:
del real_data['train']
del real_data['test']
del real_data['sales']

In [7]:
# make all column names lowercase for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name].columns = [col.lower() for col in table.columns]
for table_name, table in synthetic_data.items():
    synthetic_data[table_name].columns = [col.lower() for col in table.columns]

# rename date column to sale_date for all tables in real and synthetic data
for table_name, table in real_data.items():
    real_data[table_name] = table.rename(columns={'date': 'sale_date'})
for table_name, table in synthetic_data.items():
    synthetic_data[table_name] = table.rename(columns={'date': 'sale_date'})

# add index column of 1 to n to sales table in real and synthetic data
for table_name, table in real_data.items():
    if table_name == 'sales':
        real_data[table_name]['index'] = range(1, len(table) + 1)

for table_name, table in synthetic_data.items():
    if table_name == 'sales':
        synthetic_data[table_name]['index'] = range(1, len(table) + 1)

In [14]:
metadata = Metadata()
metadata.add_table(
    name = 'store',
    data = real_data['store'],
    primary_key='store',
    # fields_metadata=store_fields_metadata,
)

In [15]:
metadata_name = '../models/rossman-gretelai-1.0-metadata.json'
metadata.to_json(metadata_name)

In [16]:
with open(metadata_name) as metadata_file:
    metadata = json.load(metadata_file)

In [17]:
evaluate(
    synthetic_data,
    real_data,
    aggregate=False,
    metadata=metadata)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.893693,0.893693,0.0,1.0,MAXIMIZE,None
1,CSTest,Chi-Squared,0.962791,0.962791,0.0,1.0,MAXIMIZE,None


In [18]:
report = QualityReport()
report.generate(real_data, synthetic_data, metadata)

Creating report:   0%|          | 0/5 [00:00<?, ?it/s]/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
Creating report: 100%|██████████| 5/5 [00:00<00:00, 20.01it/s]



Overall Quality Score: 84.3%

Properties:
Column Shapes: 91.89%
Column Pair Trends: 76.72%
Parent Child Relationships: NaN


/Users/martinjurkovic/.pyenv/versions/3.8.12/envs/ds_project/lib/python3.8/site-packages/sdmetrics/reports/multi_table/quality_report.py:130: RuntimeWarning: Mean of empty slice
  self._property_breakdown[prop] = np.nanmean(prop_scores) if (


In [19]:
report.get_details(
    property_name='Column Shapes',
    table_name='store')

,Table,Column,Metric,Quality Score
0,store,competitiondistance,KSComplement,0.933675
1,store,competitionopensincemonth,KSComplement,0.967207
2,store,competitionopensinceyear,KSComplement,0.959142
3,store,promo2,KSComplement,0.606308
4,store,promo2sinceweek,KSComplement,0.941352
5,store,promo2sinceyear,KSComplement,0.954472
6,store,storetype,TVComplement,0.948243
7,store,assortment,TVComplement,0.971361
8,store,promointerval,TVComplement,0.988277


In [20]:
fig = report.get_visualization(
    property_name='Column Shapes',
    table_name='store')

fig.show()

In [21]:
fig = report.get_visualization(
    property_name='Column Pair Trends',
    table_name='store')

fig.show()

In [23]:
plot_all_columns(real_data, synthetic_data, metadata)

Could not plot store.store
sdtype of type 'id' not recognized.
